## ✒️ Code Modules and Helpful Functions

In [ ]:
import numpy as np,pandas as pd 
import seaborn as sns,pylab as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import OneHotEncoder,StandardScaler,RobustScaler
from sklearn.metrics import mean_squared_error,median_absolute_error,mean_absolute_error
from sklearn.metrics import r2_score,explained_variance_score
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.ensemble import BaggingRegressor,AdaBoostRegressor,ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor,RadiusNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression,SGDRegressor
from sklearn.linear_model import Ridge, RidgeCV,BayesianRidge
from sklearn.linear_model import HuberRegressor,TheilSenRegressor,RANSACRegressor
from sklearn.neural_network import MLPRegressor

In [ ]:
def regression(regressor,x_train,x_test,y_train):
    regressor.fit(x_train,y_train)    
    y_train_reg=regressor.predict(x_train)
    y_test_reg=regressor.predict(x_test)    
    return y_train_reg,y_test_reg
def scores(regressor,y_train,y_test,y_train_reg,y_test_reg):
    regressor_list.append(str(regressor))    
    ev_train_c=explained_variance_score(y_train,y_train_reg)
    ev_train.append(ev_train_c)
    ev_test_c=explained_variance_score(y_test,y_test_reg)
    ev_test.append(ev_test_c)    
    r2_train_c=r2_score(y_train,y_train_reg)
    r2_train.append(r2_train_c)
    r2_test_c=r2_score(y_test,y_test_reg)
    r2_test.append(r2_test_c)    
    mse_train_c=mean_squared_error(y_train,y_train_reg)
    mse_train.append(mse_train_c)
    mse_test_c=mean_squared_error(y_test,y_test_reg)
    mse_test.append(mse_test_c)
    mae_train_c=mean_absolute_error(y_train,y_train_reg)
    mae_train.append(mae_train_c)
    mae_test_c=mean_absolute_error(y_test,y_test_reg)
    mae_test.append(mae_test_c)     
    mdae_train_c=median_absolute_error(y_train,y_train_reg)
    mdae_train.append(mdae_train_c)
    mdae_test_c=median_absolute_error(y_test,y_test_reg)
    mdae_test.append(mdae_test_c)
    print(30*"_"); print(str(regressor)); print(30*"_")
    print("EV score. Train: ",ev_train_c)
    print("EV score. Test: ",ev_test_c); print(10*"-")
    print("R2 score. Train: ",r2_train_c)
    print("R2 score. Test: ",r2_test_c); print(10*"-")
    print("MSE score. Train: ",mse_train_c)
    print("MSE score. Test: ",mse_test_c); print(10*"-")
    print("MAE score. Train: ",mae_train_c)
    print("MAE score. Test: ",mae_test_c); print(10*"-")
    print("MdAE score. Train: ",mdae_train_c)
    print("MdAE score. Test: ",mdae_test_c)

## ✒️ Input Preprocessing

In [ ]:
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')
train.tail().T[:20]

In [ ]:
train.describe().columns

In [ ]:
# counting NaN values
train_nan=train.isnull().sum()[train.isnull().sum()!=0]
train_nan_list=train_nan[train_nan>100].index.values.tolist()
# deleting columns with a lot of NaN
for element in train_nan_list:
    del train[element]; del test[element]
train.isnull().sum()[train.isnull().sum()!=0]    

In [ ]:
# Fill in NaN values
train_cat_nan_list=['Electrical','MasVnrType','BsmtQual','BsmtCond',
                    'BsmtExposure','BsmtFinType1','BsmtFinType2',
                    'GarageType','GarageFinish','GarageQual','GarageCond']
train_num_nan_list=['MasVnrArea','GarageYrBlt']
for x in train_cat_nan_list:
    most_common=pd.get_dummies(train[x]).sum()\
    .sort_values(ascending=False).index[0]
    train[x]=train[x].fillna(most_common)
for y in train_num_nan_list:
    train[y]=train[y].fillna(train[y].median())
train.isnull().sum()[train.isnull().sum()!=0]

In [ ]:
test.isnull().sum()[test.isnull().sum()!=0]

In [ ]:
# Fill in NaN values
test_cat_nan_list=['MSZoning','Utilities','Exterior1st','Exterior2nd', 
                   'MasVnrType','BsmtQual','BsmtCond',
                   'BsmtExposure','BsmtFinType1','BsmtFinType2',
                   'KitchenQual','Functional','SaleType',
                   'GarageType','GarageFinish','GarageQual','GarageCond']
test_num_nan_list=['MasVnrArea','BsmtFinSF1','BsmtFinSF2',
                   'BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath',
                   'GarageYrBlt','GarageCars','GarageArea']
for x in test_cat_nan_list:
    most_common=pd.get_dummies(test[x]).sum()\
    .sort_values(ascending=False).index[0]
    test[x]=test[x].fillna(most_common)
for y in test_num_nan_list:
    test[y]=test[y].fillna(test[y].median())
test.isnull().sum()[test.isnull().sum()!=0]    

In [ ]:
# factorizing categorical features
object_list=train.dtypes[train.dtypes=='object']\
.index.values.tolist()
print(object_list)

In [ ]:
for element in object_list:
    train_categories=pd.factorize(train[element])
    test_categories=pd.factorize(test[element])
    train[element]=train_categories[0]
    test[element]=test_categories[0]

In [ ]:
for element in test.columns.values.tolist():
    train[element]=train[element].astype('int')
    test[element]=test[element].astype('int')
(test.dtypes=='int64').sum()==len(test.dtypes)

In [ ]:
# displaying correlation
pearson=train.corr(method='pearson')
corr_with_prices=pearson.iloc[-1][:-1]
corr_with_prices[abs(corr_with_prices).argsort()[::-1]][:30]

In [ ]:
# creating the feature list
features_list=corr_with_prices[abs(corr_with_prices)\
.argsort()[::-1]][:30].index.values.tolist()
print(features_list)
target_train=train['SalePrice'].values
features_train=train[features_list].values
features_test=test[features_list].values

In [ ]:
# data scaling
scale=RobustScaler()
features_train=scale.fit_transform(features_train)
features_test=scale.transform(features_test)

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.figure(figsize=(12,7))
plt.hist(target_train,bins=200,normed=True,alpha=.7)
plt.xlabel("Prices")
plt.title('Housing Data');

In [ ]:
X_train,X_test,y_train,y_test=\
train_test_split(features_train,target_train,
                 test_size=.2,random_state=1)
X_train.shape,X_test.shape

## ✒️ Regressors

In [ ]:
regressor_list,ev_train,ev_test=[],[],[]
r2_train,r2_test,mse_train,mse_test=[],[],[],[]
mae_train,mae_test,mdae_train,mdae_test=[],[],[],[]

### Adaptive Boosting and Random Forest Regressors

In [ ]:
y_train_abr,y_test_abr=regression(AdaBoostRegressor(),
                                  X_train,X_test,y_train)
y_train_rfr,y_test_rfr=regression(RandomForestRegressor(),
                                  X_train,X_test,y_train)
scores(AdaBoostRegressor(),y_train,y_test,y_train_abr,y_test_abr)
scores(RandomForestRegressor(),y_train,y_test,y_train_rfr,y_test_rfr)

In [ ]:
plt.figure(figsize=(12,7))
plt.plot(y_test[1:50],'-o',color='black',label='Real Data',
         markersize=7,markerfacecolor="None",markeredgewidth=2)
plt.plot(y_test_abr[1:50],'-o',label='Ada Boost',
        markersize=7, markerfacecolor="None",markeredgewidth=2)
plt.plot(y_test_rfr[1:50],'-o',label='Random Forest',
        markersize=7, markerfacecolor="None",markeredgewidth=2)
plt.legend(); plt.title("Predictions vs Real Data");

### Gradient Boosting and Bagging Regressors

In [ ]:
y_train_gbr,y_test_gbr=regression(GradientBoostingRegressor(),
                                  X_train,X_test,y_train)
y_train_br,y_test_br=regression(BaggingRegressor(),
                                X_train,X_test,y_train)
scores(GradientBoostingRegressor(),
       y_train,y_test,y_train_gbr,y_test_gbr)
scores(BaggingRegressor(),
       y_train,y_test,y_train_br,y_test_br)

In [ ]:
plt.figure(figsize=(12,7))
plt.plot(y_test[1:50],'-o',color='black',label='Real Data',
         markersize=7,markerfacecolor="None",markeredgewidth=2)
plt.plot(y_test_gbr[1:50],'-o',label='Gradient Boosting',
        markersize=7, markerfacecolor="None",markeredgewidth=2)
plt.plot(y_test_br[1:50],'-o',label='Bagging',
        markersize=7, markerfacecolor="None",markeredgewidth=2)
plt.legend(); plt.title("Predictions vs Real Data"); 

### Decision Tree and Extra Trees Regressors

In [ ]:
y_train_dtr,y_test_dtr=regression(DecisionTreeRegressor(),
                                  X_train,X_test,y_train)
y_train_etr,y_test_etr=regression(ExtraTreesRegressor(),
                                  X_train,X_test,y_train)
scores(DecisionTreeRegressor(),
       y_train,y_test,y_train_dtr,y_test_dtr)
scores(ExtraTreesRegressor(),
       y_train,y_test,y_train_etr,y_test_etr)

In [ ]:
plt.figure(figsize=(12,7))
plt.plot(y_test[1:50],'-o',color='black',label='Real Data',
         markersize=7,markerfacecolor="None",markeredgewidth=2)
plt.plot(y_test_dtr[1:50],'-o',label='Decision Tree',
         markersize=7, markerfacecolor="None",markeredgewidth=2)
plt.plot(y_test_etr[1:50],'-o',label='Extra Trees',
        markersize=7,markerfacecolor="None",markeredgewidth=2)
plt.legend(); plt.title("Predictions vs Real Data");

### K Neighbors and Radius Neighbors Regressors

In [ ]:
y_train_knr,y_test_knr=regression(KNeighborsRegressor(n_neighbors=3), 
                                  X_train,X_test,y_train)
y_train_rnr,y_test_rnr=regression(RadiusNeighborsRegressor(radius=6.35), 
                                  X_train,X_test,y_train)
scores(KNeighborsRegressor(n_neighbors=2),
       y_train,y_test,y_train_knr,y_test_knr)
scores(RadiusNeighborsRegressor(radius=6.35),
       y_train,y_test,y_train_rnr,y_test_rnr)

In [ ]:
plt.figure(figsize=(12,7))
plt.plot(y_test[1:50],'-o',color='black',label='Real Data',
         markersize=7,markerfacecolor="None",markeredgewidth=2)
plt.plot(y_test_knr[1:50],'-o',label='K Neighborse',
         markersize=7,markerfacecolor="None",markeredgewidth=2)
plt.plot(y_test_rnr[1:50],'-o',label='Radius Neighbors',
         markersize=7,markerfacecolor="None",markeredgewidth=2)
plt.legend(); plt.title("Predictions vs Real Data");

## ✒️ Evaluation Metrics

In [ ]:
df_results=pd.DataFrame({'regressor':regressor_list,
                         'ev_train':ev_train,'ev_test':ev_test,
                         'r2_train':r2_train,'r2_test':r2_test,
                         'mse_train':mse_train,'mse_test':mse_test,
                         'mae_train':mae_train,'mae_test':mae_test,
                         'mdae_train':mdae_train,'mdae_test':mdae_test})

In [ ]:
print(df_results['regressor'][5])

In [ ]:
df_results[['mae_train','mae_test','mdae_train','mdae_test']]\
.plot(figsize=(12,4));

## ✒️ Tuning Algorithms

### Gradient Boosting Regressor

In [ ]:
param_grid_gbr={'learning_rate':[.1,.05],'max_depth':[4,5], 
                'n_estimators':range(120,241,30)}
gridsearch_gbr=\
GridSearchCV(GradientBoostingRegressor(),param_grid_gbr,n_jobs=5)\
.fit(X_train,y_train)
gridsearch_gbr.best_params_

In [ ]:
regressor_gbr=\
GradientBoostingRegressor(learning_rate=.05,max_depth=4,
                          n_estimators=gridsearch_gbr.best_params_['n_estimators'],
                          min_samples_leaf=1,min_samples_split=2)
y_train_gbr_best,y_test_gbr_best=\
regression(regressor_gbr,X_train,X_test,y_train)
scores(regressor_gbr,y_train,y_test,y_train_gbr_best,y_test_gbr_best)

### Bagging Regressor

In [ ]:
param_grid_br=\
{'base_estimator':[DecisionTreeRegressor(),RandomForestRegressor(),
                   GradientBoostingRegressor()],
'n_estimators':range(90,151,30)}
gridsearch_br=GridSearchCV(BaggingRegressor(),param_grid_br,n_jobs=5)\
.fit(X_train,y_train)
gridsearch_br.best_params_

In [ ]:
regressor_br=\
BaggingRegressor(GradientBoostingRegressor(),
                 n_estimators=gridsearch_br.best_params_['n_estimators'])
y_train_br_best,y_test_br_best=\
regression(regressor_br,X_train,X_test,y_train)
scores(regressor_br,y_train,y_test,y_train_br_best,y_test_br_best)

### Random Forest Regressor

In [ ]:
param_grid_rfr={'n_estimators':range(150,301,30)}
gridsearch_rfr=GridSearchCV(RandomForestRegressor(),param_grid_rfr,n_jobs=5)\
.fit(X_train,y_train)
gridsearch_rfr.best_params_

In [ ]:
regressor_rfr=\
RandomForestRegressor(n_estimators=gridsearch_rfr.best_params_['n_estimators'])
y_train_rfr_best,y_test_rfr_best=\
regression(regressor_rfr,X_train,X_test,y_train)
scores(regressor_rfr,y_train,y_test,y_train_rfr_best,y_test_rfr_best)

### Extra Trees Regressor

In [ ]:
param_grid_etr={'n_estimators':range(150,301,30)}
gridsearch_etr=GridSearchCV(ExtraTreesRegressor(),param_grid_etr,n_jobs=10)\
.fit(X_train,y_train)
gridsearch_etr.best_params_

In [ ]:
regressor_etr=\
ExtraTreesRegressor(n_estimators=gridsearch_etr.best_params_['n_estimators'])
y_train_etr_best,y_test_etr_best=\
regression(regressor_etr,X_train,X_test,y_train)
scores(regressor_etr,y_train,y_test,y_train_etr_best,y_test_etr_best)

## ✒️ Predictions

In [ ]:
reg=GradientBoostingRegressor(learning_rate=.05,max_depth=4,
                              n_estimators=210, 
                              min_samples_leaf=1,
                              min_samples_split=2)
reg.fit(features_train,target_train)
target_train_predict=reg.predict(features_train)
target_test_predict=reg.predict(features_test)

In [ ]:
target_predict=["{0:.2f}".format(x) for x in target_test_predict]
submission=pd.DataFrame({"Id": test['Id'],"SalePrice":target_predict})
print(submission[0:5])
submission.to_csv('kaggle_housing.csv',index=False)

## ✒️  Features' Importance

In [ ]:
df_importance=pd.DataFrame(columns=range(30))
df_importance.loc[0]=\
AdaBoostRegressor().fit(X_train,y_train).feature_importances_
df_importance.loc[1]=\
RandomForestRegressor().fit(X_train,y_train).feature_importances_
df_importance.loc[2]=\
GradientBoostingRegressor().fit(X_train,y_train).feature_importances_
df_importance.loc[3]=\
DecisionTreeRegressor().fit(X_train,y_train).feature_importances_
df_importance.loc[4]=\
ExtraTreesRegressor().fit(X_train,y_train).feature_importances_

In [ ]:
plt.figure(figsize=(12,7))
plt.plot(df_importance.loc[0],'-v',color='black',label='AdaBoost',
         markersize=7,markerfacecolor="None",markeredgewidth=2)
plt.plot(df_importance.loc[1],'-v',label='Random Forest',
         markersize=7,markerfacecolor="None",markeredgewidth=2)
plt.plot(df_importance.loc[2],'-v',label='Gradient Boosting',
         markersize=7,markerfacecolor="None",markeredgewidth=2)
plt.plot(df_importance.loc[3],'-v',label='Decision Tree',
         markersize=7,markerfacecolor="None",markeredgewidth=2)
plt.plot(df_importance.loc[4],'-v',label='Extra Trees',
         markersize=7,markerfacecolor="None",markeredgewidth=2)
plt.legend(); plt.title("Importance of the Features");

In [ ]:
column_pairs=dict(zip(list(range(30)),list(features_list)))
index_pairs=dict(zip(list(range(5)), 
                     list(['AdaBoost','Random Forest','Gradient Boosting', 
                           'Decision Tree','Extra Trees'])))
df_importance2=df_importance.rename(index=index_pairs,columns=column_pairs)

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(df_importance2,cmap='Greens');